In [1]:
import pandas as pd
import numpy as np

In [7]:
#Rough 
!wget http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-s-all.gz
!gunzip ppdb-2.0-s-all.gz

--2021-07-21 07:57:30--  http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-s-all.gz
Resolving nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)... 158.130.57.54
Connecting to nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)|158.130.57.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 567671280 (541M) [application/x-gzip]
Saving to: ‘ppdb-2.0-s-all.gz’

ppdb-2.0-s-all.gz   100%[===================>] 541.37M   111MB/s    in 5.2s    

2021-07-21 07:57:35 (104 MB/s) - ‘ppdb-2.0-s-all.gz’ saved [567671280/567671280]



In [56]:
# One Time Initialisation for Papraphrase Augmentation

!pip install numpy requests nlpaug
import nltk
nltk.download('averaged_perceptron_tagger')
import nlpaug.augmenter.word as naw
skip_words = ['pm']
paraphrase_nlpaug = naw.SynonymAug(aug_src='ppdb', model_path='./ppdb-2.0-s-all', stopwords= skip_words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [83]:
# One Time Initialisation for Parrot Augmentation
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-ffvdq1i0
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-ffvdq1i0
     |████████████████████████████████| 2.5 MB 27.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 38.1 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=af9ad6338d65c024cc0e9f3726656e3eeea15a8178c735e4eaf204247b499490
  Stored in directory: /tmp/pip-ephem-wheel-cache-lx1futky/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.wh

In [68]:
def paraphrase_aug(data):
  
  result = []

  for text in data:
    aug_list = []
    try:
      aug_list = paraphrase_nlpaug.augment(text, n=5)
      print(len(aug_list))
    except:
      print('exception caught')
      aug_list.append(text)
    
    result.append(aug_list)

  return result

In [84]:
def parrot_aug(data):
  
  result = []

  for text in data:
    aug_list = []

    try:
      para_phrases = parrot.augment(input_phrase=text, 
                               diversity_ranker="levenshtein",
                               do_diverse=True, 
                               max_return_phrases = 20, 
                               max_length=32, 
                               adequacy_threshold = 0.90, 
                               fluency_threshold = 0.90)
      if para_phrases is None:
        aug_list.append(text)
      else:
        for para_phrase in para_phrases:
          aug_text = para_phrase[0]
          aug_list.append(aug_text)
    except:
      aug_list.append(text)
    
    result.append(aug_list)

  return result

In [80]:
def backtranslate_aug(data):
  
  result = []

  for text in data:
    aug_list = []
    try:
      aug_list = paraphrase_nlpaug.augment(text, n=5)
      print(len(aug_list))
    except:
      print('exception caught')
      aug_list.append(text)
    
    result.append(aug_list)

  return result

In [75]:
def augment(techniques, data):
    df = pd.DataFrame(data, columns=['utterences'])
    prev_key = 'utterences'
    for technique in techniques:
        result = []
        
        if(technique == 'paraphrase'):
            result = paraphrase_aug(df[prev_key])
        elif(technique == 'parrot'):
            result = parrot_aug(df[prev_key])
        elif(technique == 'backtranslate'):
            result = backtranslate_aug(df[prev_key])
            
        aug_df = createDf(df[prev_key], result, prev_key, technique)
        aug_df.drop_duplicates([technique], inplace=True)
        df = pd.merge(df, aug_df, how='inner', on = prev_key)
        print(df.columns)
        prev_key = technique
    
    return df

In [47]:
def createDf(data, result, colName1, colName2):
    col1 = []
    col2 = []
    for i, aug in enumerate(result):
        for j in aug:
            col1.append(data[i])
            col2.append(j)
    
    df = pd.DataFrame({colName1: col1, colName2: col2})
    return df

In [90]:
data = ['The quick brown fox jumps over the lazy dog', '4:33 pm every wednesday']
techniques = ['paraphrase', 'parrot']
df = augment(techniques, data)

5
5
Index(['utterences', 'paraphrase'], dtype='object')
Index(['utterences', 'paraphrase', 'parrot'], dtype='object')


In [92]:
print(df.shape)
df.head(19)

(18, 3)


,utterences,paraphrase,parrot
0,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox steps over the lazy dog an...
1,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the brown rapid fox climbs over the lazy dog
2,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
3,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
4,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
5,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
6,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
7,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
8,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox climbs over the lazy dog's...
9,The quick brown fox jumps over the lazy dog,The rapid brown fox climbs over the lazy dog,the rapid brown fox steps over the lazy dog
